In [1]:
# for correct relative imports
import sys; sys.path.append("../var_dgm")

In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import DataLoader, TensorDataset

from diffusers import DDPMScheduler
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pprint

from sklearn.preprocessing import StandardScaler

from var_dgm import TimeGrad
from var_dgm.basic_models import HistoricalSimulation, VarCov
from var_dgm.stat_tests import generate_report
from var_dgm.utils import seed_everything, compute_individual_returns, compute_portfolio_returns, estimate_var_es_torch

In [3]:
sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")

In [4]:
DATA_FOLDER = "../../data/"
df = pd.read_csv(DATA_FOLDER + "complete_stocks.csv")
df["Date"] = pd.to_datetime(df["Date"])

In [5]:
len(df["Ticker"].unique())

89

## 5%

In [6]:
res_timeGrad = list()
res_hist = list()
res_varcov = list()
alpha=0.05
RANDOM_STATE = 12

for i in range(5):
    # one complete cycle
    seed_everything(RANDOM_STATE + i)
    n_stocks = 10
    tickers = np.random.choice(df["Ticker"].unique(), n_stocks, replace=False)
    weights = 1/n_stocks
    print("Portfolio = {0}".format(" + ".join([f"{weights} * {i}"for i in tickers])))
    df_copy = df.loc[df["Ticker"].isin(tickers)].copy(deep=True)

    df_returns = compute_individual_returns(df_copy)
    df_returns = compute_portfolio_returns(df_returns)

    univariate_target = df_returns["Return_Target"]

    univariate_target = univariate_target.values[1:]
    train_size = df_returns[df_returns.Date <= "2022-06-01"].index[-1] + 1
    test_size = len(univariate_target) - train_size
    train = univariate_target[:train_size]

    ss = StandardScaler()
    train_scaled = torch.tensor(ss.fit_transform(train.reshape(-1, 1)), dtype=torch.float32)

    seed_everything(RANDOM_STATE)
    context_size = 90
    num_train_samples = 3000
    train_data = torch.zeros(num_train_samples, context_size, train_scaled.shape[1])
    train_target = torch.zeros(num_train_samples, 1, train_scaled.shape[1])
    train_idx = np.random.choice(np.arange(context_size, train_scaled.shape[0]), num_train_samples, replace=False)
    
    for i in tqdm(range(num_train_samples)):
        idx = train_idx[i]
        train_context = train_scaled[idx-context_size:idx]
        target_obs = train_scaled[idx]
        train_data[i] = train_context
        train_target[i] = target_obs

    # Create DataLoader for ease of torch training
    train_loader = DataLoader(TensorDataset(train_data, train_target), batch_size=128, shuffle=False)


    temp = torch.tensor(ss.transform(univariate_target.reshape(-1, 1)))
    test_data_context = torch.zeros(test_size, context_size, temp.shape[1])
    test_data_real = torch.zeros(test_size, 1, temp.shape[1])
    for i in range(test_size):
        idx = i + train_size
        test_data_context[i] = temp[idx-context_size:idx]
        test_data_real[i] = temp[idx]

    seed_everything(RANDOM_STATE)
    sheduler = DDPMScheduler(num_train_timesteps=46, beta_end=0.35878774019831244, clip_sample=False)
    model = TimeGrad(1, 1, hidden_size=50, num_layers=2, scheduler=sheduler, num_inference_steps=46)
    
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0008985916117829954)
    device = "mps"
    n_epochs = 31
    model.to(device);

    model.fit(train_loader, optimizer, n_epochs, device)


    seed_everything(RANDOM_STATE)
    VaR_TimeGrad = torch.zeros(test_data_real.shape[0])
    ES_TimeGrad = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_TimeGrad[i], ES_TimeGrad[i] = estimate_var_es_torch(model, test, alpha=alpha, n_samples=500, device="mps")

    res_timeGrad.append(generate_report(test_data_real.flatten(), VaR_TimeGrad, ES_TimeGrad, alpha=alpha))

    hist_sim = HistoricalSimulation(alpha=alpha)

    VaR_histSim = torch.zeros(test_data_real.shape[0])
    ES_histSim = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_histSim[i], ES_histSim[i] = hist_sim.predict(test)
    
    res_hist.append(generate_report(test_data_real.flatten(), VaR_histSim, ES_histSim, alpha=alpha))
    
    var_cov = VarCov(alpha=alpha)
    
    VaR_varCov = torch.zeros(test_data_real.shape[0])
    ES_varCov = torch.zeros(test_data_real.shape[0])
    
    for i in tqdm(range(test_data_real.shape[0])):
        test = test_data_context[[i]]
        VaR_varCov[i], ES_varCov[i] = var_cov.predict(test)
    
    res_varcov.append(generate_report(test_data_real.flatten(), VaR_varCov, ES_varCov, alpha=alpha))

    print(res_timeGrad[-1], res_hist[-1], res_varcov[-1])

Portfolio = 0.1 * INTC + 0.1 * DHR + 0.1 * COP + 0.1 * MRK + 0.1 * HON + 0.1 * T + 0.1 * MSFT + 0.1 * AXP + 0.1 * PEP + 0.1 * SYK


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

Epochs:   0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

...

Epochs:   0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

{'Kupicks POF': 0.012838967431808393, 'Haas TBF': 0.03996308855456492, 'Acerbi Szekely 1': 1.6945527791976929, 'Acerbi Szekely 2': 0.002139586955308914} {'Kupicks POF': 0.15902475231004007, 'Haas TBF': 0.7240770913383486, 'Acerbi Szekely 1': 2.011117935180664, 'Acerbi Szekely 2': 0.003555006580427289} {'Kupicks POF': 0.027280750843974515, 'Haas TBF': 0.6011210401926721, 'Acerbi Szekely 1': 1.9982155561447144, 'Acerbi Szekely 2': 0.0027752993628382683}


In [7]:
print(np.array([list(i.values()) for i in res_timeGrad]).mean(axis=0))
print(np.array([list(i.values()) for i in res_hist]).mean(axis=0))
print(np.array([list(i.values()) for i in res_varcov]).mean(axis=0))

[0.41711071 0.42433338 1.94789007 0.00463044]
[0.47226877 0.29528797 2.00618958 0.00414766]
[0.20164341 0.26640992 2.02576871 0.00348413]
